
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/corazzon/finance-data-analysis/blob/main/5.2%20%EC%97%85%EC%A2%85%20%ED%85%8C%EB%A7%88%20%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EC%A0%84%EC%B2%98%EB%A6%AC-output.ipynb)


## 업종/테마 전처리 미션확인

* 팀장님이 우리 회사와 유사한 업종과 테마로 사업을 하고 있는 종목을 수집해서 분석해 달라는 미션을 주셨습니다.
* 코스피에서 해당 업종 혹은 테마의 시가총액이 가장 많은 종목을 찾아주세요.
* 해당 업종/테마의 거래량이 가장 많은 종목을 찾아주세요.
* 최근 52주 최고가에서 현재가 기준으로 가장 손실이 크게 난 종목은 어떤 종목일까요?
* 거래대금이 20위 안에 있는 코스피 종목 중에 "PER"가 가장 낮은 종목은 무엇일까요?


### 주요 업종/테마

* 실습에 참고해 보세요.
* 제약 업종 : https://finance.naver.com/sise/sise_group_detail.nhn?type=upjong&no=35
* 의료기기 : https://finance.naver.com/sise/sise_group_detail.nhn?type=theme&no=175
* 코로나19(덱사메타손) : https://finance.naver.com/sise/sise_group_detail.nhn?type=theme&no=442
* 제약업체 : https://finance.naver.com/sise/sise_group_detail.nhn?type=theme&no=172
* 바이오시밀러(복제 바이오의약품) : https://finance.naver.com/sise/sise_group_detail.nhn?type=theme&no=241

In [1]:
# 라이브러리 로드하기 : 데이터 분석을 위한 pandas, 수치계산을 위한 numpy
import pandas as pd
import numpy as np

In [2]:
# 4.업종/테마주 수집시에 저장해 두었던 csv 파일을 다시 읽어옵니다.
# 'upjong_no_35_연-월-일.csv'
# 종목코드는 꼭 문자형태로 불러옵니다. dtype={"종목코드":np.object}
file_name = "upjong_no_35_2021-01-09.csv"
raw = pd.read_csv(file_name, dtype={"종목코드":np.object})
raw.shape

(139, 53)

In [3]:
# 미리보기 합니다.
raw.head()

,종목명,현재가,전일비,등락률,매수호가,매도호가,거래량,거래대금,전일거래량,조회일자,...,PERlEPS(2019.12),PBRlBPS (2019.12),배당수익률l2020.12,PERlEPS(2020.06),PBRlBPS (2020.06),배당수익률l2020.06,PERlEPS(2018.12),PBRlBPS (2018.12),PERlEPS(2020.08),PBRlBPS (2020.08)
0,바이넥스 *,29250.0,4050.0,+16.07%,29200.0,29250.0,21887861.0,642337.0,698250.0,2021-01-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,하이텍팜 *,24250.0,3350.0,+16.03%,24200.0,24250.0,994336.0,23768.0,220832.0,2021-01-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,셀트리온헬스케어 *,162900.0,8200.0,+5.30%,162800.0,162900.0,3369110.0,537535.0,2685377.0,2021-01-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,큐브앤컴퍼니 *,1435.0,65.0,+4.74%,1430.0,1435.0,826705.0,1151.0,293147.0,2021-01-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,한국파마 *,23800.0,700.0,+3.03%,23750.0,23800.0,1228182.0,29754.0,371596.0,2021-01-09,...,37.84배 l 629원,"5.70배 l 4,178원",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 전체 컬럼 출력하기

In [4]:
# 전체 컬럼이 출력되도록 display 설정을 변경합니다.
pd.options.display.max_columns = None

In [5]:
# 전체 컬럼이 보이는지 확인합니다.
raw.head()

,종목명,현재가,전일비,등락률,매수호가,매도호가,거래량,거래대금,전일거래량,조회일자,종목명_전처리,종목코드,매출액,영업이익,당기순이익,영업이익률,순이익률,ROE(지배주주),부채비율,당좌비율,유보율,EPS(원),PER(배),BPS(원),PBR(배),주당배당금(원),시가배당률(%),배당성향(%),시가총액,시가총액순위,상장주식수,액면가l매매단위,외국인한도주식수(A),외국인보유주식수(B),외국인소진율(B/A),투자의견l목표주가,52주최고l최저,PERlEPS(2020.09),추정PERlEPS,PBRlBPS (2020.09),배당수익률,동일업종 PER,동일업종 등락률,PERlEPS(2019.12),PBRlBPS (2019.12),배당수익률l2020.12,PERlEPS(2020.06),PBRlBPS (2020.06),배당수익률l2020.06,PERlEPS(2018.12),PBRlBPS (2018.12),PERlEPS(2020.08),PBRlBPS (2020.08)
0,바이넥스 *,29250.0,4050.0,+16.07%,29200.0,29250.0,21887861.0,642337.0,698250.0,2021-01-09,바이넥스,053030,364.0,113.0,44.0,31.04,12.09,NaN,NaN,NaN,NaN,NaN,NaN,5843.0,5.01,NaN,NaN,NaN,"9,287 억원",코스닥 64위,31751048,500원 l 1주,31751048,2187070,6.89%,"4.00매수 l 36,000","39,400 l 6,090",66.48배 l 440원,N/A l N/A,"5.13배 l 5,705원",NaN,109.73배,+0.84%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,하이텍팜 *,24250.0,3350.0,+16.03%,24200.0,24250.0,994336.0,23768.0,220832.0,2021-01-09,하이텍팜,106190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,719 억원",코스닥 540위,7088782,500원 l 1주,7088782,2974752,41.96%,N/A l N/A,"23,750 l 6,610",N/A l -794원,N/A l N/A,"1.79배 l 13,574원",NaN,109.73배,+0.84%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,셀트리온헬스케어 *,162900.0,8200.0,+5.30%,162800.0,162900.0,3369110.0,537535.0,2685377.0,2021-01-09,셀트리온헬스케어,091990,5071.0,1102.0,651.0,21.73,12.84,NaN,NaN,NaN,NaN,581.0,280.24,12731.0,12.80,NaN,NaN,NaN,"24조 7,298 억원",코스닥 1위,151809822,"1,000원 l 1주",151809822,26075470,17.18%,"4.00매수 l 136,939","175,738 l 49,132","100.87배 l 1,615원","87.00배 l 1,864원","13.30배 l 12,252원",NaN,109.73배,+0.84%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,큐브앤컴퍼니 *,1435.0,65.0,+4.74%,1430.0,1435.0,826705.0,1151.0,293147.0,2021-01-09,큐브앤컴퍼니,043090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,468 억원,코스닥 1251위,32590313,500원 l 1주,32590313,199199,0.61%,N/A l N/A,"4,450 l 1,125",N/A l -201원,N/A l N/A,1.87배 l 768원,NaN,109.73배,+0.84%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,한국파마 *,23800.0,700.0,+3.03%,23750.0,23800.0,1228182.0,29754.0,371596.0,2021-01-09,한국파마,032300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2751.0,8.65,NaN,NaN,NaN,"2,596 억원",코스닥 354위,10906701,500원 l 1주,10906701,9821,0.09%,N/A l N/A,"32,850 l 13,100",NaN,N/A l N/A,NaN,NaN,109.73배,+0.84%,37.84배 l 629원,"5.70배 l 4,178원",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# DataFrame의 column만 따로 봅니다.
raw.columns

Index(['종목명', '현재가', '전일비', '등락률', '매수호가', '매도호가', '거래량', '거래대금', '전일거래량',
       '조회일자', '종목명_전처리', '종목코드', '매출액', '영업이익', '당기순이익', '영업이익률', '순이익률',
       'ROE(지배주주)', '부채비율', '당좌비율', '유보율', 'EPS(원)', 'PER(배)', 'BPS(원)',
       'PBR(배)', '주당배당금(원)', '시가배당률(%)', '배당성향(%)', '시가총액', '시가총액순위', '상장주식수',
       '액면가l매매단위', '외국인한도주식수(A)', '외국인보유주식수(B)', '외국인소진율(B/A)', '투자의견l목표주가',
       '52주최고l최저', 'PERlEPS(2020.09)', '추정PERlEPS', 'PBRlBPS (2020.09)',
       '배당수익률', '동일업종 PER', '동일업종 등락률', 'PERlEPS(2019.12)',
       'PBRlBPS (2019.12)', '배당수익률l2020.12', 'PERlEPS(2020.06)',
       'PBRlBPS (2020.06)', '배당수익률l2020.06', 'PERlEPS(2018.12)',
       'PBRlBPS (2018.12)', 'PERlEPS(2020.08)', 'PBRlBPS (2020.08)'],
      dtype='object')

## 결측치

In [7]:
True + False + True

2

In [8]:
# 결측치의 합계를 구합니다.
raw.isnull().sum()

종목명                    0
현재가                    0
전일비                    0
등락률                    0
매수호가                   0
매도호가                   0
거래량                    0
거래대금                   0
전일거래량                  0
조회일자                   0
종목명_전처리                0
종목코드                   0
매출액                  113
영업이익                 113
당기순이익                118
영업이익률                113
순이익률                 118
ROE(지배주주)            139
부채비율                 139
당좌비율                 139
유보율                  139
EPS(원)               123
PER(배)               123
BPS(원)               101
PBR(배)               101
주당배당금(원)             139
시가배당률(%)             139
배당성향(%)              139
시가총액                   0
시가총액순위                 0
상장주식수                  0
액면가l매매단위               0
외국인한도주식수(A)            0
외국인보유주식수(B)            0
외국인소진율(B/A)            0
투자의견l목표주가              0
52주최고l최저               0
PERlEPS(2020.09)      15
추정PERlEPS              0
PBRlBPS (2020.09)     15


In [9]:
# 결측치의 비율을 구합니다.
raw.isnull().mean() * 100

종목명                    0.000000
현재가                    0.000000
전일비                    0.000000
등락률                    0.000000
매수호가                   0.000000
매도호가                   0.000000
거래량                    0.000000
거래대금                   0.000000
전일거래량                  0.000000
조회일자                   0.000000
종목명_전처리                0.000000
종목코드                   0.000000
매출액                   81.294964
영업이익                  81.294964
당기순이익                 84.892086
영업이익률                 81.294964
순이익률                  84.892086
ROE(지배주주)            100.000000
부채비율                 100.000000
당좌비율                 100.000000
유보율                  100.000000
EPS(원)                88.489209
PER(배)                88.489209
BPS(원)                72.661871
PBR(배)                72.661871
주당배당금(원)             100.000000
시가배당률(%)             100.000000
배당성향(%)              100.000000
시가총액                   0.000000
시가총액순위                 0.000000
상장주식수                  0.000000
액면가l매매단위

In [10]:
# 결측치 제거 전에 행과 열의 수를 봅니다.
raw.shape

(139, 53)

In [11]:
# 전체가 결측치인 컬럼은 삭제 합니다.
df = raw.dropna(axis=1, how="all").copy()
df.shape

(139, 45)

In [12]:
# 삭제된 컬럼의 수를 계산해 봅니다.
raw.shape[1] - df.shape[1]

8

In [13]:
# info를 통해 결측치 데이터 타입을 확인합니다.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 45 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   종목명                139 non-null    object 
 1   현재가                139 non-null    float64
 2   전일비                139 non-null    float64
 3   등락률                139 non-null    object 
 4   매수호가               139 non-null    float64
 5   매도호가               139 non-null    float64
 6   거래량                139 non-null    float64
 7   거래대금               139 non-null    float64
 8   전일거래량              139 non-null    float64
 9   조회일자               139 non-null    object 
 10  종목명_전처리            139 non-null    object 
 11  종목코드               139 non-null    object 
 12  매출액                26 non-null     float64
 13  영업이익               26 non-null     float64
 14  당기순이익              21 non-null     float64
 15  영업이익률              26 non-null     float64
 16  순이익률               21 non-

## 파생변수 만들기와 텍스트 데이터 다루기
<img src="https://pandas.pydata.org/docs/_images/05_newcolumn_1.svg">

* 코스피/코스닥 구분 만들기
* 시가총액 순위 분리하기
* 액면가l매매단위
* 52주최고l최저

### 코스피/코스닥 컬럼 만들기

In [14]:
# "코스피/코스닥" 컬럼을 만듭니다.
#  * 는 코스닥 종목입니다. str.endswith 를 사용합니다.
df.loc[df["종목명"].str.endswith("*"), "코스피/코스닥"] = "코스닥"
df.loc[~df["종목명"].str.endswith("*"), "코스피/코스닥"] = "코스피"

In [15]:
df["코스피/코스닥"].tail()

134    코스닥
135    코스피
136    코스닥
137    코스피
138    코스닥
Name: 코스피/코스닥, dtype: object

In [16]:
# "코스피/코스닥" 빈도수를 확인합니다.
df["코스피/코스닥"].value_counts()

코스닥    82
코스피    57
Name: 코스피/코스닥, dtype: int64

###  시가총액 수치형태로 만들기

In [17]:
# or를 의미하는 | 는 엔터키 위에 있는 키를 shift키를 누르면 나옵니다.
# df['시가총액(억원)']
df["시가총액(억원)"] = df["시가총액"].str.replace("억원", "")
df["시가총액(억원)"] = df["시가총액(억원)"].str.replace("조|,", "")
df["시가총액(억원)"] = df["시가총액(억원)"].str.replace(" ", "")
df["시가총액(억원)"] = df["시가총액(억원)"].str.strip()
df["시가총액(억원)"] = df["시가총액(억원)"].astype(int)
df["시가총액(억원)"]

0        9287
1        1719
2      247298
3         468
4        2596
        ...  
134      2714
135     34246
136      5494
137      9926
138     17280
Name: 시가총액(억원), Length: 139, dtype: int64

* [정규 표현식 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EC%A0%95%EA%B7%9C_%ED%91%9C%ED%98%84%EC%8B%9D)

In [18]:
# 정규표현식을 사용한 숫자 추출

df["시가총액(억원)"] = df["시가총액"].str.replace("[^0-9]", "").astype(int)
df["시가총액(억원)"].head(1)

0    9287
Name: 시가총액(억원), dtype: int64

### 시가총액 순위 수치데이터로 만들기

In [19]:
# split을 사용하는 방법
# '시가총액순위(전처리)' column에서도 '위'를 빼고 수치데이터로 변경합니다.
df["시가총액순위(전처리)"] =  df["시가총액순위"].str.split(" ", expand=True)[1]
df["시가총액순위(전처리)"] = df["시가총액순위(전처리)"].str.replace("위", "").astype(int)
df["시가총액순위(전처리)"]

0        64
1       540
2         1
3      1251
4       354
       ... 
134     348
135      86
136     138
137     196
138      24
Name: 시가총액순위(전처리), Length: 139, dtype: int64

In [20]:
# 정규표현식을 사용하는 방법도 있습니다. 
# 숫자를 제외한 문자는 빈문자""로 변경하도록 합니다.
df["시가총액순위(전처리)"] = df["시가총액순위"].str.replace("[^0-9]", "").astype(int)

### 52주최고l최저 

In [21]:
# '52주최고l최저' => 52주최고, 52주최저 두 개의  컬럼으로 나누어 전처리 합니다.
df["52주최고"] = df["52주최고l최저"].str.split("  l  ", expand=True)[0]
df["52주최저"] = df["52주최고l최저"].str.split("  l  ", expand=True)[1]

In [22]:
# 일부 column들을 수치형 데이터로 바꿀수 있도록 ','와 같은 문자들을 없애고
# 새로운 column에 수치형 데이터로 입력합니다.
# 이런 수치형 데이터의 column들로 새로운 데이터 column들을 생성합니다.
df["52주최고"] = df["52주최고"].str.replace(",", "").astype(int)
df["52주최저"] = df["52주최저"].str.replace(",", "").astype(int)

In [23]:
# 52주차액: 52주최고 - 52주최저 
# 52주차액비율 : 52주최고 / 52주최저 
df["52주차액"] = df["52주최고"] - df["52주최저"]
df["52주차액비율"] = df["52주최고"] / df["52주최저"]

In [24]:
# 현재-최고가 : 현재가 - 52주최고
# 현재/최고가비율 : 현재가 / 52주최고
df["현재-최고가"] = df["현재가"] - df["52주최고"]
df["현재/최고가비율"] = df["현재가"] / df["52주최고"]

In [25]:
df[["종목명", "52주차액", "52주차액비율", "52주최고", "52주최저", "현재-최고가", "현재/최고가비율"]].head()

,종목명,52주차액,52주차액비율,52주최고,52주최저,현재-최고가,현재/최고가비율
0,바이넥스 *,33310,6.469622,39400,6090,-10150.0,0.742386
1,하이텍팜 *,17140,3.593041,23750,6610,500.0,1.021053
2,셀트리온헬스케어 *,126606,3.576854,175738,49132,-12838.0,0.926948
3,큐브앤컴퍼니 *,3325,3.955556,4450,1125,-3015.0,0.322472
4,한국파마 *,19750,2.507634,32850,13100,-9050.0,0.724505


### 투자의견l목표주가 astype과 to_numeric

In [26]:
type(np.nan)

float

In [27]:
# '투자의견l목표주가' => 투자의견, 매수매도, 목표주가 컬럼으로 나누어 전처리 합니다.
# df["투자의견"]
# df["매수매도"]
# df["목표주가"]
df["목표주가"] = df["투자의견l목표주가"].str.split("  l  ", expand=True)[1]
df["목표주가"] = df["목표주가"].replace("N/A", np.nan).str.replace(",", "").astype(float)
df["목표주가"]

0       36000.0
1           NaN
2      136939.0
3           NaN
4           NaN
         ...   
134         NaN
135     25000.0
136         NaN
137         NaN
138         NaN
Name: 목표주가, Length: 139, dtype: float64

In [28]:
df["투자의견"] = df["투자의견l목표주가"].str.split("  l  ", expand=True)[0]
df["투자의견"] = df["투자의견"].replace("N/A", np.nan)
df["매수매도"] = df["투자의견"].str.replace("[0-9.]", "")
df["투자의견"] = df["투자의견"].str.replace("[^0-9.]", "").astype(float)
df["투자의견"]

0      4.0
1      NaN
2      4.0
3      NaN
4      NaN
      ... 
134    NaN
135    3.0
136    NaN
137    NaN
138    NaN
Name: 투자의견, Length: 139, dtype: float64

In [29]:
# df["투자의견"].astype(float)

In [30]:
# 투자의견의 수치만 pd.to_numeric 으로 숫자로 만들기
df["투자의견"] = pd.to_numeric(df["투자의견"])

In [31]:
# 투자의견 컬럼만 가져와서 describe로 기술통계 값을 봅니다.
df["투자의견"].describe()

count    40.000000
mean      3.844750
std       0.406618
min       2.000000
25%       4.000000
50%       4.000000
75%       4.000000
max       4.000000
Name: 투자의견, dtype: float64

### 데이터 타입 변경하기
* "동일업종 PER 컬럼"에서 "배" 문자를 제거하고 수치 변수로 변환해 주세요.
* 전처리 결괏값은 df["동일업종 PER(배)"]라는 컬럼에 담아주세요

In [32]:
# "동일업종 PER"
df["동일업종 PER"]

0      109.73배
1      109.73배
2      109.73배
3      109.73배
4      109.73배
        ...   
134    109.73배
135    109.73배
136    109.73배
137    109.73배
138    109.73배
Name: 동일업종 PER, Length: 139, dtype: object

In [33]:
# "동일업종 PER 컬럼"에서 "배" 문자를 제거하고 수치 변수로 변환해 주세요.

df["동일업종 PER"] = df["동일업종 PER"].str.replace("배", "")

In [34]:
# str.replace, astype를 이용하면 쉽게 해결할 수 있습니다.
df["동일업종 PER"] = df["동일업종 PER"].astype(float)
df["동일업종 PER"]

0      109.73
1      109.73
2      109.73
3      109.73
4      109.73
        ...  
134    109.73
135    109.73
136    109.73
137    109.73
138    109.73
Name: 동일업종 PER, Length: 139, dtype: float64

## filter와 melt로 여러 컬럼 전처리하기

### PER와 EPS 

#### PER 주가수익비율

* "이 주식은 비싼가, 주식시장이 거품인가" 판단하는 시금석
* PER = 주식가격 / 주당순이익(EPS)

물건을 살 때 값을 얼마나 줘야 좋은 건지 판단할 기준은 수없이 많다. 내게 얼마나 절실한가. 이 물건이 얼마나 유용하고 값어치가 있나. 혹은 어느 정도나 오래 사용할 수 있나. 물건에 대한 다른 사람들의 평판은 어떤가. 물건이 내게 주는 수익은 어느 정도인가…. 생각해보면 쉽게 떠오르는 이런저런 이유들은 어려운 주식투자나, 투자결정에도 그대로 적용된다.

이런 점에서 흔히 투자자들 사이에서 `퍼(PER)`로 불리는 **주가수익비율(Price earning ratio, 이하 PER)**은 어떤 회사의 주식가치, 더 나아가 전체 주식시장의 가치가 고평가 됐는지 가늠할 수 있는 유용한 잣대다. **PER은 현재 시장에서 매매되는 특정회사의 주식가격을 주당순이익으로 나눈 값을 말한다.** 한 주에 만 원하는 회사주식이 1년에 주당 1000원의 순이익을 낸다면? PER은 10이 된다. 이 수치가 회사의 가치를 어떻게 반영한다는 얘기일까?

* 출처 [네이버 지식백과]주가수익비율(PER) - "이 주식은 비싼가, 주식시장이 거품인가" 판단하는 시금석 (금융사전, 김태근)
* https://terms.naver.com/entry.nhn?docId=3571127&cid=58781&categoryId=58781

#### EPS 주당순이익

* EPS =  순이익(당기순이익) / 그 기업이 발행한 총 주식수

주당순이익(EPS)은 기업이 벌어들인 순이익(당기순이익)을 그 기업이 발행한 총 주식수로 나눈 값으로, 1주당 이익을 얼마나 창출하였느냐를 나타내는 지표이다. 즉 해당 회사가 1년간 올린 수익에 대한 주주의 몫을 나타내는 지표라 할 수 있다. 또한 주당순이익은 주가수익비율(PER)계산의 기초가 되기도 한다.

EPS가 높을수록 주식의 투자 가치는 높다고 볼 수 있으며, 그만큼 해당 회사의 경영실적이 양호하다는 뜻이다. 따라서 배당 여력도 많으므로 주가에 긍정적인 영향을 미친다.

EPS는 당기순이익 규모가 늘면 높아지게 되고, 전환사채의 주식전환이나 증자로 주식수가 많아지면 낮아지게 된다. 특히 최근 주식시장의 패턴이 기업의 수익성을 중시하는 쪽으로 바뀌면서 EPS의 크기가 중요시되고 있다.

* 출처 : [네이버 지식백과] 주당순이익(EPS) (시사상식사전, pmg 지식엔진연구소)
* https://terms.naver.com/entry.nhn?docId=74510&cid=43667&categoryId=43667

### filter 를 사용해 특정 텍스트가 들어가는 컬럼 찾기
* [정규 표현식 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EC%A0%95%EA%B7%9C_%ED%91%9C%ED%98%84%EC%8B%9D)

In [35]:
# df.filter(regex=)를 통해 "PERlEPS" 가 들어가는 컬럼만 가져옵니다.
# 특정 컬럼을 가져오고 종목코드를 추가해 줍니다.
df_per_eps = df.filter(regex="^PERlEPS").copy()
df_per_eps["종목코드"] = df["종목코드"]
df_per_eps

,PERlEPS(2020.09),PERlEPS(2019.12),PERlEPS(2020.06),PERlEPS(2018.12),PERlEPS(2020.08),종목코드
0,66.48배 l 440원,NaN,NaN,NaN,NaN,053030
1,N/A l -794원,NaN,NaN,NaN,NaN,106190
2,"100.87배 l 1,615원",NaN,NaN,NaN,NaN,091990
3,N/A l -201원,NaN,NaN,NaN,NaN,043090
4,NaN,37.84배 l 629원,NaN,NaN,NaN,032300
...,...,...,...,...,...,...
134,N/A l -17원,NaN,NaN,NaN,NaN,074430
135,67.86배 l 868원,NaN,NaN,NaN,NaN,003090
136,NaN,N/A l -354원,NaN,NaN,NaN,321550
137,"32.99배 l 1,852원",NaN,NaN,NaN,NaN,033270


### melt로 tidy data 만들기

<img src="https://pandas.pydata.org/docs/_images/reshaping_melt.png" width="600">

* https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-by-melt
* [Tidy Data 란?](https://vita.had.co.nz/papers/tidy-data.pdf)


In [36]:
# melt를 통해 종목코드를 녹여냅니다.
df_per_eps_melt = df_per_eps.melt(id_vars="종목코드")
df_per_eps_melt

,종목코드,variable,value
0,053030,PERlEPS(2020.09),66.48배 l 440원
1,106190,PERlEPS(2020.09),N/A l -794원
2,091990,PERlEPS(2020.09),"100.87배 l 1,615원"
3,043090,PERlEPS(2020.09),N/A l -201원
4,032300,PERlEPS(2020.09),NaN
...,...,...,...
690,074430,PERlEPS(2020.08),NaN
691,003090,PERlEPS(2020.08),NaN
692,321550,PERlEPS(2020.08),NaN
693,033270,PERlEPS(2020.08),NaN


In [37]:
# 종목코드별로 value_counts를 통해  PERlEPS 행이 몇 개인지 봅니다.
df_per_eps_melt["종목코드"].value_counts()

000640    5
293480    5
106190    5
298060    5
003220    5
         ..
002210    5
001540    5
260660    5
086060    5
009420    5
Name: 종목코드, Length: 139, dtype: int64

### 결측치 데이터 제거
<img src="https://pandas.pydata.org/docs/_images/03_subset_rows.svg">

In [38]:
# PERlEPS 데이터프레임에서 결측치 데이터 제거
# 여기에서 기간별로 생긴 행은 이전 실습에서 concat으로 데이터를 합칠 때 
# 컬럼의 이름이 다르면 새로운 컬럼으로 생성하기 때문입니다.
df_pe = df_per_eps_melt.dropna()
df_pe

,종목코드,variable,value
0,053030,PERlEPS(2020.09),66.48배 l 440원
1,106190,PERlEPS(2020.09),N/A l -794원
2,091990,PERlEPS(2020.09),"100.87배 l 1,615원"
3,043090,PERlEPS(2020.09),N/A l -201원
5,068270,PERlEPS(2020.09),"99.19배 l 3,700원"
...,...,...,...
275,321550,PERlEPS(2019.12),N/A l -354원
277,323990,PERlEPS(2019.12),N/A l -117원
362,330350,PERlEPS(2020.06),"21.33배 l 1,036원"
537,298060,PERlEPS(2018.12),"N/A l -5,321원"


In [39]:
# variable 컬럼명을 rename을 통해 "PERlEPS"로 변경합니다.
df_pe = df_pe.rename(columns={"variable": "PERlEPS"})
df_pe

,종목코드,PERlEPS,value
0,053030,PERlEPS(2020.09),66.48배 l 440원
1,106190,PERlEPS(2020.09),N/A l -794원
2,091990,PERlEPS(2020.09),"100.87배 l 1,615원"
3,043090,PERlEPS(2020.09),N/A l -201원
5,068270,PERlEPS(2020.09),"99.19배 l 3,700원"
...,...,...,...
275,321550,PERlEPS(2019.12),N/A l -354원
277,323990,PERlEPS(2019.12),N/A l -117원
362,330350,PERlEPS(2020.06),"21.33배 l 1,036원"
537,298060,PERlEPS(2018.12),"N/A l -5,321원"


In [40]:
# df_pe에서 PER, EPS 컬럼을 따로 생성합니다.
df_pe["PER"] = df_pe["value"].str.split("  l  ", expand=True)[0]
df_pe["EPS"] = df_pe["value"].str.split("  l  ", expand=True)[1]

In [41]:
# PER, EPS 컬럼의 N/A 값을 결측치로 만듭니다.
df_pe["PER"] = df_pe["PER"].replace("N/A", np.nan)
df_pe["EPS"] = df_pe["EPS"].replace("N/A", np.nan)

In [42]:
#  PER, EPS 값 중 둘 다 결측치는 notnull 혹은 isnull을 사용해 제거합니다.
df_pe = df_pe[df_pe["PER"].notnull() | df_pe["EPS"].notnull()].copy()

In [43]:
# PER(배) 컬럼을 만들고 수치데이터 형태로 변환합니다.
df_pe["PER(배)"] = df_pe["PER"].str.replace("[^0-9.]", "").astype(float)

In [44]:
# EPS(원) 컬럼을 만들고 수치데이터 형태로 변환합니다.
df_pe["EPS(원)"] = df_pe["EPS"].str.replace("원|,", "").astype(int)

In [45]:
# 필요한 컬럼만 따로 가져옵니다.
df_pe = df_pe[["종목코드", "PERlEPS", "PER(배)", "EPS(원)"]].copy()
df_pe

,종목코드,PERlEPS,PER(배),EPS(원)
0,053030,PERlEPS(2020.09),66.48,440
1,106190,PERlEPS(2020.09),NaN,-794
2,091990,PERlEPS(2020.09),100.87,1615
3,043090,PERlEPS(2020.09),NaN,-201
5,068270,PERlEPS(2020.09),99.19,3700
...,...,...,...,...
275,321550,PERlEPS(2019.12),NaN,-354
277,323990,PERlEPS(2019.12),NaN,-117
362,330350,PERlEPS(2020.06),21.33,1036
537,298060,PERlEPS(2018.12),NaN,-5321


### 중복 데이터 확인

In [46]:
# "종목코드"기준으로 중복된 데이터가 있는지 찾습니다.
df_pe[df_pe.duplicated("종목코드")]

,종목코드,PERlEPS,PER(배),EPS(원)


In [47]:
# 만약 중복데이터가 있다면 다음과 같은 방법으로 제거합니다.
# 추출했던 원래 데이터프레임 df와 다시 merge 로 합치기 위해 
# drop_duplicates 를 사용해 종목코드가 중복되는 행은 제거합니다.
# "종목코드", "PERlEPS" 로 정렬을 하고 
# drop_duplicates 로 제거시 keep="first" 로 위에 있는 데이터를 남깁니다.
df_pe = df_pe.sort_values(["종목코드", "PERlEPS"])
df_pe = df_pe.drop_duplicates(["종목코드", "PERlEPS"])
df_pe

,종목코드,PERlEPS,PER(배),EPS(원)
96,000020,PERlEPS(2020.09),18.60,1027
8,000100,PERlEPS(2020.09),29.55,2596
23,000105,PERlEPS(2020.09),26.81,2596
87,000220,PERlEPS(2020.09),35.24,403
51,000225,PERlEPS(2020.09),34.24,403
...,...,...,...,...
275,321550,PERlEPS(2019.12),NaN,-354
277,323990,PERlEPS(2019.12),NaN,-117
160,326030,PERlEPS(2019.12),NaN,-1100
362,330350,PERlEPS(2020.06),21.33,1036


In [48]:
df_pe["종목코드"].value_counts().unique()

array([1])

### PBRlBPS 전처리


#### PBR(주가순자산비율, price-to-book ratio, P/B ratio)

* PBR = 주가/BPS(주당순자산가치)
* 주가는 그 회사의 종합적인 평가이므로 주주 소유분을 초과한 부분은 모두 그 회사의 잠재적인 프리미엄이 되기 때문에 **경영의 종합력이 뛰어나면 뛰어날수록 배율이 높아진다**고 할 수 있다. 일반적으로 이 지표는 PER과 함께 사용되는데, 이는 주가순자산배율이 그 회사를 정적인 상태 즉 스토크(stock)면에서 보고 있는데 반하여 주가수익률은 동적인 상태 즉 플로(flow)면에서 보고 있으므로 두 가지 지표가 서로 보완관계에 있다.

* 다만 이 주가 순자산배율에도 **주주소유분이 장부가격에 의해 계산되고 있어 반드시 정확하다고 할 수 없고 또 그 계산기준이 전기의 결산기가 된다는 등의 난점이 있다.**
* 출처 : [주가순자산비율 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EC%A3%BC%EA%B0%80%EC%88%9C%EC%9E%90%EC%82%B0%EB%B9%84%EC%9C%A8)

#### BPS(주당순자산가치, Book-value Per Share)

* 주당순자산가치 = 기업의 순자산/ 발행 주식수
* 주당순자산가치는, '청산가치' 라고도 불린다. 그 이유는 현재 시점에서 기업의 활동을 중단시키고 그 부를 모든 주주들에게 나눠줄 경우 한주당 얼마씩이 돌아가는가를 나타내는 수치이기 때문이다. 보통 주당순자산가치라는 말은 잘 쓰지 않고 BPS라고 하므로 이하 BPS로 쓴다.

* BPS가 높다는 것은 자기자본의 비중이 크거나 회사 가치가 높다는 것을 나타낸다. BPS에는 주가 정보가 고려되어 있지 않으므로, 이 회사의 주가가 자산 가치에 비해 얼마나 저평가 혹은 고평가되어 있는지 판단하기 위해 PBR이라는 값을 사용한다.

* PBR 값은 BPS 값으로 현재 주가를 나눈 것이다. BPS가 1만원인 위 사례에서 주가가 5,000원이라면 PBR은 0.5가 되고, 주가가 20,000원이라면 PBR은 2가 된다. 즉 전자의 경우 주가는 실제의 자산에 비해 저평가되어 되어 있는 것이고, 후자의 경우 주가는 실제 자산에 비해 고평가 되어 있는 것이다.

* 출처 : [주당순자산가치 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EC%A3%BC%EB%8B%B9%EC%88%9C%EC%9E%90%EC%82%B0%EA%B0%80%EC%B9%98)


In [49]:
# filter를 사용해 PBRlBPS 컬럼 가져옵니다.
# 종목코드 추가합니다.
df_pbr_bps = df.filter(regex="PBRlBPS").copy()
df_pbr_bps["종목코드"] = df["종목코드"]
df_pbr_bps

,PBRlBPS (2020.09),PBRlBPS (2019.12),PBRlBPS (2020.06),PBRlBPS (2018.12),PBRlBPS (2020.08),종목코드
0,"5.13배 l 5,705원",NaN,NaN,NaN,NaN,053030
1,"1.79배 l 13,574원",NaN,NaN,NaN,NaN,106190
2,"13.30배 l 12,252원",NaN,NaN,NaN,NaN,091990
3,1.87배 l 768원,NaN,NaN,NaN,NaN,043090
4,NaN,"5.70배 l 4,178원",NaN,NaN,NaN,032300
...,...,...,...,...,...,...
134,5.12배 l 604원,NaN,NaN,NaN,NaN,074430
135,"3.86배 l 15,248원",NaN,NaN,NaN,NaN,003090
136,NaN,"6.42배 l 3,598원",NaN,NaN,NaN,321550
137,"3.30배 l 18,496원",NaN,NaN,NaN,NaN,033270


In [50]:
# melt로 tidy data 만듭니다.
df_pbr_bps_melt = df_pbr_bps.melt(id_vars="종목코드").copy()
df_pbr_bps_melt

,종목코드,variable,value
0,053030,PBRlBPS (2020.09),"5.13배 l 5,705원"
1,106190,PBRlBPS (2020.09),"1.79배 l 13,574원"
2,091990,PBRlBPS (2020.09),"13.30배 l 12,252원"
3,043090,PBRlBPS (2020.09),1.87배 l 768원
4,032300,PBRlBPS (2020.09),NaN
...,...,...,...
690,074430,PBRlBPS (2020.08),NaN
691,003090,PBRlBPS (2020.08),NaN
692,321550,PBRlBPS (2020.08),NaN
693,033270,PBRlBPS (2020.08),NaN


In [51]:
# 결측치를 제거합니다.
df_pb = df_pbr_bps_melt.dropna()
df_pb

,종목코드,variable,value
0,053030,PBRlBPS (2020.09),"5.13배 l 5,705원"
1,106190,PBRlBPS (2020.09),"1.79배 l 13,574원"
2,091990,PBRlBPS (2020.09),"13.30배 l 12,252원"
3,043090,PBRlBPS (2020.09),1.87배 l 768원
5,068270,PBRlBPS (2020.09),"15.47배 l 23,717원"
...,...,...,...
275,321550,PBRlBPS (2019.12),"6.42배 l 3,598원"
277,323990,PBRlBPS (2019.12),"96.88배 l 2,371원"
362,330350,PBRlBPS (2020.06),"2.56배 l 8,617원"
537,298060,PBRlBPS (2018.12),"9.46배 l 5,147원"


In [52]:
# variable 컬럼명을 "PBRlBPS"로 변경합니다.
df_pb = df_pb.rename(columns={"variable":"PBRlBPS"})
df_pb

,종목코드,PBRlBPS,value
0,053030,PBRlBPS (2020.09),"5.13배 l 5,705원"
1,106190,PBRlBPS (2020.09),"1.79배 l 13,574원"
2,091990,PBRlBPS (2020.09),"13.30배 l 12,252원"
3,043090,PBRlBPS (2020.09),1.87배 l 768원
5,068270,PBRlBPS (2020.09),"15.47배 l 23,717원"
...,...,...,...
275,321550,PBRlBPS (2019.12),"6.42배 l 3,598원"
277,323990,PBRlBPS (2019.12),"96.88배 l 2,371원"
362,330350,PBRlBPS (2020.06),"2.56배 l 8,617원"
537,298060,PBRlBPS (2018.12),"9.46배 l 5,147원"


In [53]:
# PBR, BPS 컬럼을 만듭니다.
df_pb["PBR"] = df_pb["value"].str.split("  l  ", expand=True)[0]
df_pb["BPS"] = df_pb["value"].str.split("  l  ", expand=True)[1]

In [54]:
# N/A 값(문자) np.nan(결측치)로 변환합니다.
df_pb["PBR"] = df_pb["PBR"].replace("N/A", np.nan)
df_pb["BPS"] = df_pb["BPS"].replace("N/A", np.nan)

In [55]:
# PBR(배), BPS(원) 컬럼을 만듭니다.
df_pb["PBR(배)"] = df_pb["PBR"].str.replace("배", "").astype(float)
df_pb["BPS(원)"] = df_pb["BPS"].str.replace("원|,", "").astype(float)

In [56]:
df_pb["종목코드"].value_counts().unique()

array([1])

In [57]:
# "종목코드","PBRlBPS" 로 중복값이 있는지 확인합니다.
# 만약 중복값이 있다면 drop_duplicates 로 삭제합니다.
df_pb[df_pb.duplicated(["종목코드"])]

,종목코드,PBRlBPS,value,PBR,BPS,PBR(배),BPS(원)


In [58]:
df_pb = df_pb[["종목코드", "PBRlBPS", "PBR(배)", "BPS(원)"]].copy()
df_pb

,종목코드,PBRlBPS,PBR(배),BPS(원)
0,053030,PBRlBPS (2020.09),5.13,5705.0
1,106190,PBRlBPS (2020.09),1.79,13574.0
2,091990,PBRlBPS (2020.09),13.30,12252.0
3,043090,PBRlBPS (2020.09),1.87,768.0
5,068270,PBRlBPS (2020.09),15.47,23717.0
...,...,...,...,...
275,321550,PBRlBPS (2019.12),6.42,3598.0
277,323990,PBRlBPS (2019.12),96.88,2371.0
362,330350,PBRlBPS (2020.06),2.56,8617.0
537,298060,PBRlBPS (2018.12),9.46,5147.0


### 배당 수익률 전처리

In [59]:
# PER|EPS 전처리와 마찬가리로 배당수익률 데이터를 전처리 합니다.
# df.filter(regex=) 로 배당수익률이 들어가는 컬럼만 찾고 종목코드를 추가 합니다.
df_income = df.filter(regex="배당수익률").copy()

In [60]:
# 추후 병합을 위해 데이터프레임에 종목코드를 추가합니다.
df_income["종목코드"] = df["종목코드"]
df_income

,배당수익률l2020.12,배당수익률l2020.06,종목코드
0,NaN,NaN,053030
1,NaN,NaN,106190
2,NaN,NaN,091990
3,NaN,NaN,043090
4,NaN,NaN,032300
...,...,...,...
134,NaN,NaN,074430
135,NaN,NaN,003090
136,NaN,NaN,321550
137,NaN,NaN,033270


In [61]:
# melt 로 여러 컬럼의 종목코드를 행으로 녹이고 결측치를 dropna 로 제거합니다.
df_in = df_income.melt(id_vars="종목코드").copy()
df_in = df_in.dropna()
df_in

,종목코드,variable,value
10,003000,배당수익률l2020.12,0.33%
30,239610,배당수익률l2020.12,0.87%
44,084110,배당수익률l2020.12,1.32%
54,200670,배당수익률l2020.12,1.71%
65,243070,배당수익률l2020.12,0.92%
78,007370,배당수익률l2020.12,1.48%
98,260660,배당수익률l2020.12,0.69%
110,012790,배당수익률l2020.12,0.42%
223,330350,배당수익률l2020.06,0.93%


In [62]:
# 컬럼명을 "종목코드", "배당수익률기간", "배당수익률" 로 변경합니다.
df_in.columns = ["종목코드", "배당수익률기간", "배당수익률"]
df_in

,종목코드,배당수익률기간,배당수익률
10,003000,배당수익률l2020.12,0.33%
30,239610,배당수익률l2020.12,0.87%
44,084110,배당수익률l2020.12,1.32%
54,200670,배당수익률l2020.12,1.71%
65,243070,배당수익률l2020.12,0.92%
78,007370,배당수익률l2020.12,1.48%
98,260660,배당수익률l2020.12,0.69%
110,012790,배당수익률l2020.12,0.42%
223,330350,배당수익률l2020.06,0.93%


In [63]:
# 배당수익률% 컬럼을 만들어 전처리 합니다.

df_in["배당수익률%"] = df_in["배당수익률"].str.replace("%", "").astype(float)

In [64]:
# 종목코드가 종목당 2개 이상 있는 값이 있는지 확인합니다.
# 2개 이상 있다면 merge 에서 어떤 값을 사용할지 고려해야 합니다.
df_in["종목코드"].value_counts().unique()

array([1])

In [65]:
# 전처리가 끝난 배당수익률 컬럼은 제거합니다.
del df_in["배당수익률"]

In [66]:
df_in

,종목코드,배당수익률기간,배당수익률%
10,003000,배당수익률l2020.12,0.33
30,239610,배당수익률l2020.12,0.87
44,084110,배당수익률l2020.12,1.32
54,200670,배당수익률l2020.12,1.71
65,243070,배당수익률l2020.12,0.92
78,007370,배당수익률l2020.12,1.48
98,260660,배당수익률l2020.12,0.69
110,012790,배당수익률l2020.12,0.42
223,330350,배당수익률l2020.06,0.93


## merge 로 기존 데이터와 병합하기

<img src="https://pandas.pydata.org/docs/_images/08_merge_left.svg">

* concat과 merge의 차이
    * concat :  
        * axis=0 행을 기준으로 위아래로 **같은 컬럼끼리** 값을 이어 붙여 **새로운 행을 만듦**
        * axis=1 컬럼을 기준으로 **인덱스가 같은 값**을 옆으로 붙여 **새로운 컬럼을 만듦**
    * merge :
        * index 혹은 특정 컬럼 값을 기준으로 두 개의 데이터프레임을 연결
        
<img src="https://www.dofactory.com/img/sql/sql-joins.png">

* 이미지 출처 : https://www.dofactory.com/sql/join

In [67]:
df.shape

(139, 57)

In [68]:
# merge로 병합합니다.
# df와 df_pe를 병합니다. merge 전후 행과 열의 수를 확인해서 제대로 병합이 되었는지 확인합니다.
# df_merge_pe 에 할당합니다.
df_merge_pe = df.merge(df_pe, left_on="종목코드", right_on="종목코드", how="left")
df_merge_pe.head(1)

,종목명,현재가,전일비,등락률,매수호가,매도호가,거래량,거래대금,전일거래량,조회일자,종목명_전처리,종목코드,매출액,영업이익,당기순이익,영업이익률,순이익률,EPS(원)_x,PER(배)_x,BPS(원),PBR(배),시가총액,시가총액순위,상장주식수,액면가l매매단위,외국인한도주식수(A),외국인보유주식수(B),외국인소진율(B/A),투자의견l목표주가,52주최고l최저,PERlEPS(2020.09),추정PERlEPS,PBRlBPS (2020.09),동일업종 PER,동일업종 등락률,PERlEPS(2019.12),PBRlBPS (2019.12),배당수익률l2020.12,PERlEPS(2020.06),PBRlBPS (2020.06),배당수익률l2020.06,PERlEPS(2018.12),PBRlBPS (2018.12),PERlEPS(2020.08),PBRlBPS (2020.08),코스피/코스닥,시가총액(억원),시가총액순위(전처리),52주최고,52주최저,52주차액,52주차액비율,현재-최고가,현재/최고가비율,목표주가,투자의견,매수매도,PERlEPS,PER(배)_y,EPS(원)_y
0,바이넥스 *,29250.0,4050.0,+16.07%,29200.0,29250.0,21887861.0,642337.0,698250.0,2021-01-09,바이넥스,053030,364.0,113.0,44.0,31.04,12.09,NaN,NaN,5843.0,5.01,"9,287 억원",코스닥 64위,31751048,500원 l 1주,31751048,2187070,6.89%,"4.00매수 l 36,000","39,400 l 6,090",66.48배 l 440원,N/A l N/A,"5.13배 l 5,705원",109.73,+0.84%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,코스닥,9287,64,39400,6090,33310,6.469622,-10150.0,0.742386,36000.0,4.0,매수,PERlEPS(2020.09),66.48,440.0


In [69]:
df_merge_pe.shape

(139, 60)

In [70]:
# 위에서 병합한 데이터(df_merge_pe)에 PBRlBPS(df_pb) 데이터도 병합해 줍니다.
df_merge_pb = df_merge_pe.merge(df_pb, on="종목코드", how="left")
df_merge_pb.head(1)

,종목명,현재가,전일비,등락률,매수호가,매도호가,거래량,거래대금,전일거래량,조회일자,종목명_전처리,종목코드,매출액,영업이익,당기순이익,영업이익률,순이익률,EPS(원)_x,PER(배)_x,BPS(원)_x,PBR(배)_x,시가총액,시가총액순위,상장주식수,액면가l매매단위,외국인한도주식수(A),외국인보유주식수(B),외국인소진율(B/A),투자의견l목표주가,52주최고l최저,PERlEPS(2020.09),추정PERlEPS,PBRlBPS (2020.09),동일업종 PER,동일업종 등락률,PERlEPS(2019.12),PBRlBPS (2019.12),배당수익률l2020.12,PERlEPS(2020.06),PBRlBPS (2020.06),배당수익률l2020.06,PERlEPS(2018.12),PBRlBPS (2018.12),PERlEPS(2020.08),PBRlBPS (2020.08),코스피/코스닥,시가총액(억원),시가총액순위(전처리),52주최고,52주최저,52주차액,52주차액비율,현재-최고가,현재/최고가비율,목표주가,투자의견,매수매도,PERlEPS,PER(배)_y,EPS(원)_y,PBRlBPS,PBR(배)_y,BPS(원)_y
0,바이넥스 *,29250.0,4050.0,+16.07%,29200.0,29250.0,21887861.0,642337.0,698250.0,2021-01-09,바이넥스,053030,364.0,113.0,44.0,31.04,12.09,NaN,NaN,5843.0,5.01,"9,287 억원",코스닥 64위,31751048,500원 l 1주,31751048,2187070,6.89%,"4.00매수 l 36,000","39,400 l 6,090",66.48배 l 440원,N/A l N/A,"5.13배 l 5,705원",109.73,+0.84%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,코스닥,9287,64,39400,6090,33310,6.469622,-10150.0,0.742386,36000.0,4.0,매수,PERlEPS(2020.09),66.48,440.0,PBRlBPS (2020.09),5.13,5705.0


In [71]:
df_merge_pb.shape

(139, 63)

In [72]:
# 위에서 병합한 데이터(df_merge_pb)에 배당수익률(df_in) 데이터도 병합해 줍니다.
# df_final 이라는 변수에 담습니다.
df_final = df_merge_pb.merge(df_in, how="left")

In [73]:
df_final.shape

(139, 65)

In [74]:
df_final.head(1)

,종목명,현재가,전일비,등락률,매수호가,매도호가,거래량,거래대금,전일거래량,조회일자,종목명_전처리,종목코드,매출액,영업이익,당기순이익,영업이익률,순이익률,EPS(원)_x,PER(배)_x,BPS(원)_x,PBR(배)_x,시가총액,시가총액순위,상장주식수,액면가l매매단위,외국인한도주식수(A),외국인보유주식수(B),외국인소진율(B/A),투자의견l목표주가,52주최고l최저,PERlEPS(2020.09),추정PERlEPS,PBRlBPS (2020.09),동일업종 PER,동일업종 등락률,PERlEPS(2019.12),PBRlBPS (2019.12),배당수익률l2020.12,PERlEPS(2020.06),PBRlBPS (2020.06),배당수익률l2020.06,PERlEPS(2018.12),PBRlBPS (2018.12),PERlEPS(2020.08),PBRlBPS (2020.08),코스피/코스닥,시가총액(억원),시가총액순위(전처리),52주최고,52주최저,52주차액,52주차액비율,현재-최고가,현재/최고가비율,목표주가,투자의견,매수매도,PERlEPS,PER(배)_y,EPS(원)_y,PBRlBPS,PBR(배)_y,BPS(원)_y,배당수익률기간,배당수익률%
0,바이넥스 *,29250.0,4050.0,+16.07%,29200.0,29250.0,21887861.0,642337.0,698250.0,2021-01-09,바이넥스,053030,364.0,113.0,44.0,31.04,12.09,NaN,NaN,5843.0,5.01,"9,287 억원",코스닥 64위,31751048,500원 l 1주,31751048,2187070,6.89%,"4.00매수 l 36,000","39,400 l 6,090",66.48배 l 440원,N/A l N/A,"5.13배 l 5,705원",109.73,+0.84%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,코스닥,9287,64,39400,6090,33310,6.469622,-10150.0,0.742386,36000.0,4.0,매수,PERlEPS(2020.09),66.48,440.0,PBRlBPS (2020.09),5.13,5705.0,NaN,NaN


## 사용하지 않는 컬럼 제거하기 drop

In [75]:
# l 구분자가 들어가는 컬럼명만 찾습니다.
del_cols = df_final.filter(regex="l").columns
del_cols

Index(['액면가l매매단위', '투자의견l목표주가', '52주최고l최저', 'PERlEPS(2020.09)', '추정PERlEPS',
       'PBRlBPS (2020.09)', 'PERlEPS(2019.12)', 'PBRlBPS (2019.12)',
       '배당수익률l2020.12', 'PERlEPS(2020.06)', 'PBRlBPS (2020.06)',
       '배당수익률l2020.06', 'PERlEPS(2018.12)', 'PBRlBPS (2018.12)',
       'PERlEPS(2020.08)', 'PBRlBPS (2020.08)', 'PERlEPS', 'PBRlBPS'],
      dtype='object')

In [76]:
# 위에서 찾은 컬럼명을 drop으로 삭제합니다.
df_final = df_final.drop(del_cols, axis=1).copy()

In [77]:
# 결측치수가 많거나 중복된 컬럼이 있는지 확인합니다.
df_final.shape

(139, 47)

In [78]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139 entries, 0 to 138
Data columns (total 47 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   종목명          139 non-null    object 
 1   현재가          139 non-null    float64
 2   전일비          139 non-null    float64
 3   등락률          139 non-null    object 
 4   매수호가         139 non-null    float64
 5   매도호가         139 non-null    float64
 6   거래량          139 non-null    float64
 7   거래대금         139 non-null    float64
 8   전일거래량        139 non-null    float64
 9   조회일자         139 non-null    object 
 10  종목명_전처리      139 non-null    object 
 11  종목코드         139 non-null    object 
 12  매출액          26 non-null     float64
 13  영업이익         26 non-null     float64
 14  당기순이익        21 non-null     float64
 15  영업이익률        26 non-null     float64
 16  순이익률         21 non-null     float64
 17  EPS(원)_x     16 non-null     float64
 18  PER(배)_x     16 non-null     float64
 19  BPS(원)_x

In [79]:
# 전처리한 컬럼은 원본 컬럼을 제거합니다.
del_cols2 = ['종목명_전처리', '시가총액', '시가총액순위', '배당수익률기간']

In [80]:
# del_cols2 를 제거합니다.
df_final = df_final.drop(del_cols2, axis=1)

In [81]:
# 컬럼명 중 필요없는 컬럼이 없는지 확인합니다.
df_final.columns

Index(['종목명', '현재가', '전일비', '등락률', '매수호가', '매도호가', '거래량', '거래대금', '전일거래량',
       '조회일자', '종목코드', '매출액', '영업이익', '당기순이익', '영업이익률', '순이익률', 'EPS(원)_x',
       'PER(배)_x', 'BPS(원)_x', 'PBR(배)_x', '상장주식수', '외국인한도주식수(A)',
       '외국인보유주식수(B)', '외국인소진율(B/A)', '동일업종 PER', '동일업종 등락률', '코스피/코스닥',
       '시가총액(억원)', '시가총액순위(전처리)', '52주최고', '52주최저', '52주차액', '52주차액비율',
       '현재-최고가', '현재/최고가비율', '목표주가', '투자의견', '매수매도', 'PER(배)_y', 'EPS(원)_y',
       'PBR(배)_y', 'BPS(원)_y', '배당수익률%'],
      dtype='object')

In [82]:
df_final.shape

(139, 43)

## CSV 파일로 저장하기

In [83]:
# 기존 파일명을 변경해서 새로 저장할 파일명을 만듭니다.
file_name

'upjong_no_35_2021-01-09.csv'

In [84]:
# 저장할 파일명을 만들어 줍니다.
# 기존 파일명에 _ver2 를 붙여줍니다.
final_file_name = file_name.replace(".csv", "_ver2.csv")
final_file_name

'upjong_no_35_2021-01-09_ver2.csv'

In [85]:
# final_file_name 변수에 있는 파일명으로 저장합니다.
df_final.to_csv(final_file_name, index=False)

In [86]:
# final_file_name 변수에 있는 값을 읽어옵니다.
# 이 때, 종목코드는 dtype={"종목코드":np.object} 로 불러옵니다.
pd.read_csv(final_file_name, dtype={"종목코드":np.object})

,종목명,현재가,전일비,등락률,매수호가,매도호가,거래량,거래대금,전일거래량,조회일자,종목코드,매출액,영업이익,당기순이익,영업이익률,순이익률,EPS(원)_x,PER(배)_x,BPS(원)_x,PBR(배)_x,상장주식수,외국인한도주식수(A),외국인보유주식수(B),외국인소진율(B/A),동일업종 PER,동일업종 등락률,코스피/코스닥,시가총액(억원),시가총액순위(전처리),52주최고,52주최저,52주차액,52주차액비율,현재-최고가,현재/최고가비율,목표주가,투자의견,매수매도,PER(배)_y,EPS(원)_y,PBR(배)_y,BPS(원)_y,배당수익률%
0,바이넥스 *,29250.0,4050.0,+16.07%,29200.0,29250.0,21887861.0,642337.0,698250.0,2021-01-09,053030,364.0,113.0,44.0,31.04,12.09,NaN,NaN,5843.0,5.01,31751048,31751048,2187070,6.89%,109.73,+0.84%,코스닥,9287,64,39400,6090,33310,6.469622,-10150.0,0.742386,36000.0,4.0,매수,66.48,440.0,5.13,5705.0,NaN
1,하이텍팜 *,24250.0,3350.0,+16.03%,24200.0,24250.0,994336.0,23768.0,220832.0,2021-01-09,106190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7088782,7088782,2974752,41.96%,109.73,+0.84%,코스닥,1719,540,23750,6610,17140,3.593041,500.0,1.021053,NaN,NaN,NaN,NaN,-794.0,1.79,13574.0,NaN
2,셀트리온헬스케어 *,162900.0,8200.0,+5.30%,162800.0,162900.0,3369110.0,537535.0,2685377.0,2021-01-09,091990,5071.0,1102.0,651.0,21.73,12.84,581.0,280.24,12731.0,12.80,151809822,151809822,26075470,17.18%,109.73,+0.84%,코스닥,247298,1,175738,49132,126606,3.576854,-12838.0,0.926948,136939.0,4.0,매수,100.87,1615.0,13.30,12252.0,NaN
3,큐브앤컴퍼니 *,1435.0,65.0,+4.74%,1430.0,1435.0,826705.0,1151.0,293147.0,2021-01-09,043090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32590313,32590313,199199,0.61%,109.73,+0.84%,코스닥,468,1251,4450,1125,3325,3.955556,-3015.0,0.322472,NaN,NaN,NaN,NaN,-201.0,1.87,768.0,NaN
4,한국파마 *,23800.0,700.0,+3.03%,23750.0,23800.0,1228182.0,29754.0,371596.0,2021-01-09,032300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2751.0,8.65,10906701,10906701,9821,0.09%,109.73,+0.84%,코스닥,2596,354,32850,13100,19750,2.507634,-9050.0,0.724505,NaN,NaN,NaN,37.84,629.0,5.70,4178.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,아미노로직스 *,3090.0,250.0,-7.49%,3090.0,3095.0,9176444.0,29505.0,27747178.0,2021-01-09,074430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87826844,87826844,1336528,1.52%,109.73,+0.84%,코스닥,2714,348,3675,1040,2635,3.533654,-585.0,0.840816,NaN,NaN,NaN,NaN,-17.0,5.12,604.0,NaN
135,대웅,58900.0,5200.0,-8.11%,58900.0,59000.0,1444576.0,88777.0,2513749.0,2021-01-09,003090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58141980,58141980,3135970,5.39%,109.73,+0.84%,코스피,34246,86,86700,6470,80230,13.400309,-27800.0,0.679354,25000.0,3.0,중립,67.86,868.0,3.86,15248.0,NaN
136,티움바이오 *,23100.0,2150.0,-8.51%,23100.0,23150.0,1655234.0,39036.0,2065006.0,2021-01-09,321550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23783926,23783926,82902,0.35%,109.73,+0.84%,코스닥,5494,138,29650,5800,23850,5.112069,-6550.0,0.779089,NaN,NaN,NaN,NaN,-354.0,6.42,3598.0,NaN
137,유나이티드제약,61100.0,6500.0,-9.62%,61000.0,61100.0,1478006.0,94018.0,1087406.0,2021-01-09,033270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16244822,16244822,839501,5.17%,109.73,+0.84%,코스피,9926,196,112000,10750,101250,10.418605,-50900.0,0.545536,NaN,NaN,NaN,32.99,1852.0,3.30,18496.0,NaN
